# Standard ML training

In [6]:
import torch
from 

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(784, 64)
        self.fc2 = torch.nn.Linear(64, 32)
        self.fc3 = torch.nn.Linear(32, 10)

    def forward(self, x):
        x = torch.nn.functional.relu(self.fc1(x.reshape(x.size(0), 784)))
        x = torch.nn.functional.dropout(x, p=0.5, training=self.training)
        x = torch.nn.functional.relu(self.fc2(x))
        x = torch.nn.functional.log_softmax(self.fc3(x), dim=1)
        return x


In [8]:
def validate_model(model):

    model.eval()
    # Evaluate
    criterion = torch.nn.NLLLoss()
    with torch.no_grad():
        train_out = model(x_train)
        training_loss = criterion(train_out, y_train)
        training_accuracy = torch.sum(torch.argmax(train_out, dim=1) == y_train) / len(train_out)
        test_out = model(x_test)
        test_loss = criterion(test_out, y_test)
        test_accuracy = torch.sum(torch.argmax(test_out, dim=1) == y_test) / len(test_out)

    print("training loss: ", training_loss.item(),
        ", training_accuracy: ", training_accuracy.item(),
        ", test_loss: ", test_loss.item(),
        ", test_accuracy: ", test_accuracy.item()
         )

In [ ]:
# Download and load MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

model = Net()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.NLLLoss()

# Training loop
epochs = 10
for epoch in range(epochs):  
    for batch_idx, (data, target) in enumerate(train_loader):  
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    

# Federated ML training

In [ ]:
def training_round(weights):
    # Convert from numpy array to correct pytorch format
    params_dict = zip(model.state_dict().keys(), weights)
    state_dict = collections.OrderedDict({key: torch.tensor(x) for key, x in params_dict})
    model.load_state_dict(state_dict, strict=True)
    model.set_weights(weights)

    # Training loop
    local_epochs = 1
    for epoch in range(local_epochs):  
        for batch_idx, (data, target) in enumerate(train_loader):  
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

    # Convert from pytorch weights format numpy array 
    updated_weights = [val.cpu().numpy() for _, val in model.state_dict().items()]

    metadata = {
        # num_examples are mandatory
        "num_examples": len(x_train),
        "batch_size": batch_size,
        "epochs": epochs,
        "lr": lr,
    }

    return updated_weights, metadata

In [ ]:
def validate_weights(weights):
    # Convert from numpy array to correct pytorch format
    params_dict = zip(model.state_dict().keys(), weights)
    state_dict = collections.OrderedDict({key: torch.tensor(x) for key, x in params_dict})
    model.load_state_dict(state_dict, strict=True)
    model.set_weights(weights)
    model.eval()

    # Evaluate
    criterion = torch.nn.NLLLoss()
    with torch.no_grad():
        train_out = model(x_train)
        training_loss = criterion(train_out, y_train)
        training_accuracy = torch.sum(torch.argmax(train_out, dim=1) == y_train) / len(train_out)
        test_out = model(x_test)
        test_loss = criterion(test_out, y_test)
        test_accuracy = torch.sum(torch.argmax(test_out, dim=1) == y_test) / len(test_out)

    # JSON schema
    performance = {
        "training_loss": training_loss.item(),
        "training_accuracy": training_accuracy.item(),
        "test_loss": test_loss.item(),
        "test_accuracy": test_accuracy.item(),
    }

    return performance

In [ ]:
from fedn.network.clients.fedn_client import FednClient, ConnectToApiResult

fedn_client = FednClient(train_callback=training_round, validate_callback=validate_weights)

controller_url = "<paste-controller-url-here>"
token = "<paste-token-here>"

name = "<write-client-name-here>"
fedn_client.set_name(name)

client_id = str(uuid.uuid4())
fedn_client.set_client_id(client_id)

controller_config = {
    "name": name,
    "client_id": client_id,
    "package": "local",
    "preferred_combiner": "",
}

result, combiner_config = fedn_client.connect_to_api(controller_url, token, controller_config)

if result != ConnectToApiResult.Assigned:
    print("Failed to connect to API, exiting.")
    exit (1)

result: bool = fedn_client.init_grpchandler(config=combiner_config, client_name=client_id, token=token)

if not result:
    exit (1)